In [27]:
import numpy as np
from utilities import *
import control as ct
import sympy as sp


In [28]:
# !conda install -c conda-forge slycot -y


In [29]:
psi = 0.0

In [30]:
A_lin = np.array([
            [-D_vxy__CB/(m_Axy + m_RB), 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, -D_vxy__CB/(m_Axy + m_RB), 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, -D_vz__CB/(m_Az + m_RB), 0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, -D_omega_xy__CB/(I_Axy + I_RBxy + m_RB*r_z_gb__b**2), 0, 0, 0, 0, 0, -(f_g*r_z_gb__b)/(I_Axy + I_RBxy + m_RB*r_z_gb__b**2), 0, 0],
            [0, 0, 0, 0, -D_omega_xy__CB/(I_Axy + I_RBxy + m_RB*r_z_gb__b**2), 0, 0, 0, 0, 0, -(f_g*r_z_gb__b)/(I_Axy + I_RBxy + m_RB*r_z_gb__b**2), 0],
            [0, 0, 0, 0, 0, -D_omega_z__CB/(I_Az + I_RBz), 0, 0, 0, 0, 0, 0],
            [np.cos(psi), -np.sin(psi), 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
            [np.sin(psi), np.cos(psi), 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
        ])

In [31]:
B_lin = np.array([
            [1/(m_Axy + m_RB), 0, 0, 0],
            [0, 1/(m_Axy + m_RB), 0, 0],
            [0, 0, 1/(m_Az + m_RB), 0],
            [0, -r_z_tg__b/(I_Axy + I_RBxy + m_RB*r_z_gb__b**2), 0, 0],
            [r_z_tg__b/(I_Axy + I_RBxy + m_RB*r_z_gb__b**2), 0, 0, 0],
            [0, 0, 0, 1/(I_Az + I_RBz)],
            [0, 0, 0, 0],
            [0, 0, 0, 0],
            [0, 0, 0, 0],
            [0, 0, 0, 0],
            [0, 0, 0, 0],
            [0, 0, 0, 0],
        ])

In [24]:
C = np.zeros([4,12])
C[0,0],C[1,1], C[2,2], C[3,8] = 1,1,1,1
print(f"{C = }")


C = array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]])


In [25]:
# sys = ct.ss(A_lin, B_lin, np.eye(12), np.zeros((12, 4)))
# Calculate the controllability matrix
linearized_blimp_controllability_matrix = ct.ctrb(A_lin, B_lin)

print(f"{np.linalg.matrix_rank(linearized_blimp_controllability_matrix) = }")  
print(f"{A_lin.shape = }")
if np.linalg.matrix_rank(linearized_blimp_controllability_matrix) == A_lin.shape[0]:
    print("The system is controllable")
else:
    print("The system is not controllable")


np.linalg.matrix_rank(linearized_blimp_controllability_matrix) = 12
A_lin.shape = (12, 12)
The system is controllable


In [26]:
# Define state-space matrices
A = A_lin  # 12x12
B = B_lin  # 12x4
C = C      # 4x12
D = np.zeros((4, 4))  # 4x4 zero matrix
T = 0.25
# Convert to transfer function
sys_tf = ct.ss2tf(A, B, C, D)

# Print the transfer function
print(sys_tf)


ControlMIMONotImplemented: Not implemented for MIMO systems without slycot.

In [ ]:
np.linalg.inv(A)

LinAlgError: Singular matrix

In [ ]:
# define phi_alpha matrix by block
tl = A
tr = B
bl = -np.linalg.inv(C @np.linalg.inv(A)@ np.eye(A.shape[0]) @ B)@ C @ np.exp2(A* T)
br = -np.eye(C.shape[0])

# print(f"{tl.shape = }")
# print(f"{tr.shape = }")


# print(f"{bl.shape = }")
# print(f"{br.shape = }")

phi_alpha = np.block([[tl, tr], [bl, br]])

print(f"{phi_alpha = }")

LinAlgError: Singular matrix

# Testing Wardi IJRNC Paper Example for \alpha-Stability

In [42]:
# Define state-space matrices
A = np.array([[2, 1],
              [-1, -1]])

B = np.array([[0],
              [1]])

C = np.array([[-10, 1]])

D = np.array([[0]])  # Can be omitted if zero

T = .25  # Time constant

# Convert to transfer function
sys_tf = ct.ss2tf(A, B, C, D)

# Print the transfer function
print(sys_tf)



  s - 12
-----------
s^2 - s - 1



In [43]:
# sys = ct.ss(A_lin, B_lin, np.eye(12), np.zeros((12, 4)))
# Calculate the controllability matrix
example_controllability_matrix = ct.ctrb(A, B)

print(f"{np.linalg.matrix_rank(example_controllability_matrix) = }")
print(f"{A.shape = }")
if np.linalg.matrix_rank(example_controllability_matrix) == A.shape[0]:
    print("The system is controllable")
else:
    print("The system is not controllable")

np.linalg.matrix_rank(example_controllability_matrix) = 2
A.shape = (2, 2)
The system is controllable


In [44]:
# define phi_alpha matrix by block
tl = A
tr = B
bl = -np.linalg.inv(C @np.linalg.inv(A)@ np.eye(A.shape[0]) @ B)@ C @ np.exp2(A* T)
br = -np.eye(C.shape[0])

# print(f"{tl.shape = }")
# print(f"{tr.shape = }")
# print(f"{bl.shape = }")
# print(f"{br.shape = }")

phi_alpha = np.block([[tl, tr], [bl, br]])

print(f"{phi_alpha = }")

phi_alpha = array([[ 2.        ,  1.        ,  0.        ],
       [-1.        , -1.        ,  1.        ],
       [-1.1084366 , -0.92093123, -1.        ]])


In [45]:
# # define psi_alpha matrix by block
# tt = np.zeros((len(A), len(A)))
# bb = np.linalg.inv( C @ np.linalg.inv(A) @ (np.exp2(A*0.8) - np.eye(len(A))) @ B)

# # print(f"{tt.shape = }")
# # print(f"{bb.shape = }")
# psi_alpha = np.block([[tt],[bb]])
# print(f"{psi_alpha = }")

In [46]:
# work with symbolic alpha variable and get characteristic polynomial
# Define symbolic variable alpha
alpha = sp.symbols('alpha')

tl = sp.Matrix(tl)
tr = sp.Matrix(tr)
bl = sp.Matrix(bl)
br = sp.Matrix(br)

# Multiply the bottom row by alpha
bl_alpha = alpha * bl
br_alpha = alpha * br

# Construct the full block matrix with symbolic alpha
phi_alpha = sp.Matrix.vstack(
    sp.Matrix.hstack(tl, tr),
    sp.Matrix.hstack(bl_alpha, br_alpha)
)



phi_alpha

Matrix([
[                      2,                        1,          0],
[                     -1,                       -1,          1],
[-1.10843660070644*alpha, -0.920931227897791*alpha, -1.0*alpha]])

In [47]:
# Compute the characteristic polynomial
char_poly = phi_alpha.charpoly().as_expr()

# Expand the characteristic polynomial to separate terms
char_poly_expanded = sp.expand(char_poly)

# Display the results
print("Characteristic Polynomial:")
char_poly_expanded

Characteristic Polynomial:


1.0*alpha*lambda**2 - 0.0790687721022088*alpha*lambda - 1.73342585508915*alpha + 1.0*lambda**3 - 1.0*lambda**2 - 1.0*lambda

In [ ]:
# Factor the polynomial to isolate alpha
factored_poly = sp.collect(char_poly_expanded, alpha)

# Extract the coefficient of alpha and independent terms
P0 = factored_poly.coeff(alpha)
P1 = factored_poly - alpha * P0

# Display the results
print("Factored Polynomial:", factored_poly)
print("Coefficient of alpha:", P0)
print("Independent part:", P1)


# Solve for lambda in both cases
ll = sp.symbols('lambda')  # Unicode lambda
poles_P0 = sp.solve(P0, ll)
print("Poles (terms multiplied by alpha):", poles_P0)


NameError: name 'char_poly_expanded' is not defined

# Do it with Jax!

In [33]:
import jax
import jax.numpy as jnp
from jax import jit, jacfwd, lax
import numpy as np


## Cross product operator (S operator)
@jit
def S(r):
    return jnp.block([[  0,       -r[2],    r[1]],
                     [r[2],     0,      -r[0]],
                     [-r[1],    r[0],      0]])

@jit
def H(r):
    return jnp.block([[jnp.identity(3), S(r).T],
                     [jnp.zeros((3, 3)), jnp.identity(3)]])

## Rotation matrices
@jit
def R_b__n(phi, theta, psi):
    x_rot = jnp.array([[1,         0,           0],
                      [0,       jnp.cos(phi),   - jnp.sin(phi)],
                      [0,       jnp.sin(phi),    jnp.cos(phi)]])
    y_rot = jnp.array([[ jnp.cos(theta),      0,        jnp.sin(theta)],
                      [0,         1,           0],
                      [- jnp.sin(theta),     0,        jnp.cos(theta)]])
    z_rot = jnp.array([[ jnp.cos(psi),    - jnp.sin(psi),       0],
                      [ jnp.sin(psi),     jnp.cos(psi),       0],
                      [0,          0,           1]])
    return z_rot @ y_rot @ x_rot

@jit
def R_b__n_inv(phi, theta, psi):
    return jnp.array([[ jnp.cos(psi)* jnp.cos(theta), jnp.cos(theta)* jnp.sin(psi), - jnp.sin(theta)],
                     [ jnp.cos(psi)* jnp.sin(phi)* jnp.sin(theta) - jnp.cos(phi)* jnp.sin(psi), 
                      jnp.cos(phi)* jnp.cos(psi) + jnp.sin(phi)* jnp.sin(psi)* jnp.sin(theta), 
                      jnp.cos(theta)* jnp.sin(phi)],
                     [ jnp.sin(phi)* jnp.sin(psi) + jnp.cos(phi)* jnp.cos(psi)* jnp.sin(theta), 
                      jnp.cos(phi)* jnp.sin(psi)* jnp.sin(theta) - jnp.cos(psi)* jnp.sin(phi), 
                      jnp.cos(phi)* jnp.cos(theta)]])

## Transformation matrix for angular velocity
@jit
def T(phi, theta):
    return jnp.array([[1,     jnp.sin(phi)* jnp.tan(theta),      jnp.cos(phi)* jnp.tan(theta)],
                     [0,          jnp.cos(phi),                   - jnp.sin(phi)],
                     [0,     jnp.sin(phi)/jnp.cos(theta),      jnp.cos(phi)/jnp.cos(theta)]])


Henv = 0.51  # Height of the envelope (m)
Hgon = 0.04  # Height of the gondola (m)
r_z_gb__b = 0.05  # Distance from CG to CB (m)
dvt = Henv / 2 + Hgon # Distance from CB to motor thrust application point
r_z_tg__b = dvt - r_z_gb__b  # Distance from CB to thrust generation point
r_z_gb__b = 0.05
r_gb__b = jnp.array([0, 0, r_z_gb__b]).T

# Mass and inertia properties
m_RB = 0.1249  # Mass of the blimp (kg)
I_RBx = 0.005821  # Rotational inertia about x-axis (kg.m^2)
M_RB_CG = jnp.diag(jnp.array([m_RB, m_RB, m_RB, I_RBx, I_RBx, I_RBx]))# Inertia matrix for the rigid body in body frame


M_RB_CB = H(r_gb__b).T @ M_RB_CG @ H(r_gb__b)
M_A_CB = jnp.diag(jnp.array([0.0466, 0.0466, 0.0545, 0.0, 0.0, 0.0])) # Added mass and inertia matrix (assuming hydrodynamics)

# Total inertia matrix at the center of buoyancy
M_CB = M_RB_CB + M_A_CB
M_CB_inv = jnp.linalg.inv(M_CB)

# Gravitational force in the navigation frame
g_acc = 9.8  # Gravitational acceleration (m/s^2)
fg_n = m_RB * jnp.array([0, 0, g_acc]).T  # Force due to gravity

# Aerodynamic damping matrix
D_CB = jnp.diag(jnp.array([0.0125, 0.0125, 0.0480, 0.000862, 0.000862, 0.000862]))

##########################################################################################
@jit # Coriolis matrix
def C(M, nu):
    dimM = jnp.shape(M)[0]
    M11 = M[0:int(dimM/2), 0:int(dimM/2)]
    M12 = M[0:int(dimM/2), int(dimM/2):dimM]
    M21 = M[int(dimM/2):dimM, 0:int(dimM/2)]
    M22 = M[int(dimM/2):dimM, int(dimM/2):dimM]
    dimNu = jnp.shape(nu)[0]
    nu1 = nu[0:int(dimNu/2)]
    nu2 = nu[int(dimNu/2):dimNu]
    return jnp.block([[ jnp.zeros((3, 3)),    -S(M11 @ nu1 + M12 @ nu2)],
                     [-S(M11 @ nu1 + M12 @ nu2), -S(M21 @ nu1 + M22 @ nu2)]])

##########################################################################################
@jit # Dynamics function
def dynamics(state, u):
        curr_x, curr_y, curr_z, curr_vx, curr_vy, curr_vz, curr_roll, curr_pitch, curr_yaw, curr_wx, curr_wy, curr_wz = state
        tau_B = jnp.array([u[0][0],
                        u[1][0],
                        u[2][0],
                        -r_z_tg__b * u[1][0],
                        r_z_tg__b * u[0][0],
                        u[3][0]]).reshape((6,1))
        
        nu_bn_b = jnp.array([curr_vx, curr_vy, curr_vz, curr_wx, curr_wy, curr_wz]).reshape((6,1))
        fg_B = R_b__n_inv(curr_roll, curr_pitch, curr_yaw) @ fg_n
        g_CB = -jnp.block([[jnp.zeros((3, 1))],
                        [jnp.reshape(jnp.cross(r_gb__b, fg_B), (3, 1))]])
        
        x = curr_x
        y = curr_y
        z = curr_z
        vx = curr_vx
        vy = curr_vy
        vz = curr_vz
        roll = curr_roll
        pitch = curr_pitch
        yaw = curr_yaw
        wx = curr_wx
        wy = curr_wy
        wz = curr_wz


        # Update state derivatives
        eta_bn_n_dot = jnp.block([[R_b__n(roll, pitch, yaw), jnp.zeros((3, 3))],
                                [jnp.zeros((3, 3)), T(roll, pitch)]]) @ nu_bn_b

        nu_bn_b_dot = jnp.reshape(-M_CB_inv @ (C(M_CB, nu_bn_b) @ nu_bn_b + 
                            D_CB @ nu_bn_b + g_CB - tau_B), (6, 1))

        xdot = eta_bn_n_dot[0][0]
        ydot = eta_bn_n_dot[1][0]
        zdot = eta_bn_n_dot[2][0]
        vxdot = nu_bn_b_dot[0][0]
        vydot = nu_bn_b_dot[1][0]
        vzdot = nu_bn_b_dot[2][0]
        rolldot = eta_bn_n_dot[3][0]
        pitchdot = eta_bn_n_dot[4][0]
        yawdot = eta_bn_n_dot[5][0]
        wxdot = nu_bn_b_dot[3][0]
        wydot = nu_bn_b_dot[4][0]
        wzdot = nu_bn_b_dot[5][0]


        return jnp.array([xdot, ydot, zdot, vxdot, vydot, vzdot, rolldot, pitchdot, yawdot, wxdot, wydot, wzdot])

@jit # Euler integration
def integrate_dynamics(state, inputs, integration_step, integrations_int):
    def for_function(i, current_state):
        return current_state + dynamics(current_state, inputs) * integration_step
    pred_state = lax.fori_loop(0, integrations_int, for_function, state)
    return pred_state

@jit # Compute Jacobian
def euler_jacU(state, inputs, T_lookahead, integration_step, C):
    jacobianU = jacfwd(euler_pred, argnums=1)(state, inputs, T_lookahead, integration_step, C)
    return jacobianU.reshape(4,4)

@jit # Prediction with Euler integration
def euler_pred(state, inputs, T_lookahead, integration_step, C):
    integrations_int = (T_lookahead / integration_step).astype(int)
    pred_state = integrate_dynamics(state, inputs, integration_step, integrations_int)
    return C@pred_state

@jit # Compute adjusted inverse Jacobian
def euler_invjacU(state, inputs, T_lookahead, integration_step, C):
    jac = euler_jacU(state, inputs, T_lookahead, integration_step, C)
    regularization_term = 1e-9
    jac += jnp.eye(jac.shape[0]) * regularization_term
    
    cond_number = jnp.linalg.cond(jac)
    inv_jacobianU = jnp.linalg.pinv(jac)
    return inv_jacobianU, cond_number

@jit
def state_pred_full(state, inputs, T_lookahead, integration_step):
    integrations_int = (T_lookahead / integration_step).astype(int)
    pred_state = integrate_dynamics(state, inputs, integration_step, integrations_int)
    return pred_state

@jit # Compute B_lin
def state_euler_jacU_full(state, inputs, T_lookahead, integration_step):
    jacobianU = jacfwd(state_pred_full, argnums=1)(state, inputs, T_lookahead, integration_step)
    return jacobianU.squeeze()

@jit # Compute A_lin
def state_euler_jacX_full(state, inputs, T_lookahead, integration_step):
    jacobianU = jacfwd(state_pred_full, argnums=0)(state, inputs, T_lookahead, integration_step)
    return jacobianU

In [34]:
# HOVER STATE AND INPUT
hover_state = jnp.array([0., 0., -0.3, 0., 0., 0., 0., 0., 0.0, 0., 0., 0.])
hover_input = jnp.array([[0.], [0.], [0.], [0.]])
integration_step = 0.01
T_lookahead = 0.25

# Compute Jacobian
observer_matrix = np.zeros([4,12])
observer_matrix[0,0],observer_matrix[1,1], observer_matrix[2,2], observer_matrix[3,8] = 1,1,1,1
observer_matrix_jax = jnp.array(observer_matrix)
print(f"{observer_matrix_jax = }")
print(f"{hover_state = }")
print(f"{hover_input = }")
dgdu, conditionnum = euler_invjacU(hover_state, hover_input, T_lookahead, integration_step, observer_matrix_jax)

observer_matrix_jax = Array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]], dtype=float32)
hover_state = Array([ 0. ,  0. , -0.3,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
        0. ], dtype=float32)
hover_input = Array([[0.],
       [0.],
       [0.],
       [0.]], dtype=float32)


In [35]:
B_lin = np.array(state_euler_jacU_full(hover_state, hover_input, T_lookahead, integration_step).tolist())
A_lin = np.array(state_euler_jacX_full(hover_state, hover_input, T_lookahead, integration_step).tolist())
print(f"{B_lin = }")
print(f"{A_lin = }")

B_lin = array([[ 0.13762006,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.13762006,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.16384377,  0.        ],
       [ 1.16082525,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  1.16082525,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  1.34969628,  0.        ],
       [ 0.        , -1.00350368,  0.        ,  0.        ],
       [ 1.00350368,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  5.09571505],
       [ 0.        , -7.87802076,  0.        ,  0.        ],
       [ 7.87802076,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , 42.19334793]])
A_lin = array([[ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         2.47746810e-01,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  1.06499624e-02,  0.00000000e+00,
         0.00000000e+00,  9.87074687e-04,  0.00000000e+00],
       [ 0.000

In [36]:
# B_lin = array([[ 0.13762006,  0.        ,  0.        ,  0.        ],
#        [ 0.        ,  0.13762006,  0.        ,  0.        ],
#        [ 0.        ,  0.        ,  0.16384377,  0.        ],
#        [ 1.16082525,  0.        ,  0.        ,  0.        ],
#        [ 0.        ,  1.16082525,  0.        ,  0.        ],
#        [ 0.        ,  0.        ,  1.34969628,  0.        ],
#        [ 0.        , -1.00350368,  0.        ,  0.        ],
#        [ 1.00350368,  0.        ,  0.        ,  0.        ],
#        [ 0.        ,  0.        ,  0.        ,  5.09571505],
#        [ 0.        , -7.87802076,  0.        ,  0.        ],
#        [ 7.87802076,  0.        ,  0.        ,  0.        ],
#        [ 0.        ,  0.        ,  0.        , 42.19334793]])
# A_lin = array([[ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
#          2.47746810e-01,  0.00000000e+00,  0.00000000e+00,
#          0.00000000e+00,  1.06499624e-02,  0.00000000e+00,
#          0.00000000e+00,  9.87074687e-04,  0.00000000e+00],
#        [ 0.00000000e+00,  1.00000000e+00,  0.00000000e+00,
#          0.00000000e+00,  2.47746810e-01,  0.00000000e+00,
#         -1.06499624e-02,  0.00000000e+00,  0.00000000e+00,
#         -9.87074687e-04,  0.00000000e+00,  0.00000000e+00],
#        [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00,
#          0.00000000e+00,  0.00000000e+00,  2.42135480e-01,
#          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
#          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
#        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
#          9.81323361e-01,  0.00000000e+00,  0.00000000e+00,
#          0.00000000e+00,  8.32585171e-02,  0.00000000e+00,
#          0.00000000e+00,  1.18226372e-02,  0.00000000e+00],
#        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
#          0.00000000e+00,  9.81323361e-01,  0.00000000e+00,
#         -8.32585171e-02,  0.00000000e+00,  0.00000000e+00,
#         -1.18226372e-02,  0.00000000e+00,  0.00000000e+00],
#        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
#          0.00000000e+00,  0.00000000e+00,  9.35214639e-01,
#          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
#          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
#        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
#          0.00000000e+00, -2.17520189e-03,  0.00000000e+00,
#          7.05855608e-01,  0.00000000e+00,  0.00000000e+00,
#          2.22776055e-01,  0.00000000e+00,  0.00000000e+00],
#        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
#          2.17520189e-03,  0.00000000e+00,  0.00000000e+00,
#          0.00000000e+00,  7.05855608e-01,  0.00000000e+00,
#          0.00000000e+00,  2.22776055e-01,  0.00000000e+00],
#        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
#          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
#          0.00000000e+00,  0.00000000e+00,  1.00000000e+00,
#          0.00000000e+00,  0.00000000e+00,  2.45607480e-01],
#        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
#          0.00000000e+00, -1.70051400e-02,  0.00000000e+00,
#         -2.30776000e+00,  0.00000000e+00,  0.00000000e+00,
#          6.73351347e-01,  0.00000000e+00,  0.00000000e+00],
#        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
#          1.70051400e-02,  0.00000000e+00,  0.00000000e+00,
#          0.00000000e+00, -2.30776000e+00,  0.00000000e+00,
#          0.00000000e+00,  6.73351347e-01,  0.00000000e+00],
#        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
#          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
#          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
#          0.00000000e+00,  0.00000000e+00,  9.63629365e-01]])

NameError: name 'array' is not defined

In [37]:
C = np.zeros([4,12])
C[0,0],C[1,1], C[2,2], C[3,8] = 1,1,1,1
print(f"{C = }")

# sys = ct.ss(A_lin, B_lin, np.eye(12), np.zeros((12, 4)))
# Calculate the controllability matrix
linearized_blimp_controllability_matrix = ct.ctrb(A_lin, B_lin)

print(f"{np.linalg.matrix_rank(linearized_blimp_controllability_matrix) = }")  
print(f"{A_lin.shape = }")
if np.linalg.matrix_rank(linearized_blimp_controllability_matrix) == A_lin.shape[0]:
    print("The system is controllable")
else:
    print("The system is not controllable")



C = array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]])
np.linalg.matrix_rank(linearized_blimp_controllability_matrix) = 12
A_lin.shape = (12, 12)
The system is controllable


In [38]:
# Define state-space matrices
A = A_lin  # 12x12
B = B_lin  # 12x4
C = C      # 4x12
# D = np.zeros((4, 4))  # 4x4 zero matrix

In [39]:
np.exp2(A*T)

TypeError: unsupported operand type(s) for *: 'float' and 'PjitFunction'

In [40]:
# define phi_alpha matrix by block
tl = A
tr = B
bl = -np.linalg.inv(C @np.linalg.inv(A)@ np.eye(A.shape[0]) @ B)@ C @ np.exp2(A* T_lookahead)
br = -np.eye(C.shape[0])

# print(f"{tl.shape = }")
# print(f"{tr.shape = }")
# print(f"{bl.shape = }")
# print(f"{br.shape = }")

phi_alpha = np.block([[tl, tr], [bl, br]])

print(f"{phi_alpha.shape = }")

phi_alpha.shape = (16, 16)


In [41]:
# work with symbolic alpha variable and get characteristic polynomial
# Define symbolic variable alpha
alpha = sp.symbols('alpha')

tl = sp.Matrix(tl)
tr = sp.Matrix(tr)
bl = sp.Matrix(bl)
br = sp.Matrix(br)

# Multiply the bottom row by alpha
bl_alpha = alpha * bl
br_alpha = alpha * br

# Construct the full block matrix with symbolic alpha
phi_alpha = sp.Matrix.vstack(
    sp.Matrix.hstack(tl, tr),
    sp.Matrix.hstack(bl_alpha, br_alpha)
)



print(f"{phi_alpha = }")

phi_alpha = Matrix([
[                    1.0,                       0,                       0,       0.247746810317039,                       0,                       0,                       0,      0.0106499623507261,                       0,                       0,     0.00098707468714565,                      0, 0.137620061635971,                 0,                 0,                0],
[                      0,                     1.0,                       0,                       0,       0.247746810317039,                       0,     -0.0106499623507261,                       0,                       0,    -0.00098707468714565,                       0,                      0,                 0, 0.137620061635971,                 0,                0],
[                      0,                       0,                     1.0,                       0,                       0,       0.242135480046272,                       0,                       0,                       0,  

In [ ]:
phi_alpha = Matrix([
[                    1.0,                       0,                       0,       0.247746810317039,                       0,                       0,                       0,      0.0106499623507261,                       0,                       0,     0.00098707468714565,                      0, 0.137620061635971,                 0,                 0,                0],
[                      0,                     1.0,                       0,                       0,       0.247746810317039,                       0,     -0.0106499623507261,                       0,                       0,    -0.00098707468714565,                       0,                      0,                 0, 0.137620061635971,                 0,                0],
[                      0,                       0,                     1.0,                       0,                       0,       0.242135480046272,                       0,                       0,                       0,                       0,                       0,                      0,                 0,                 0, 0.163843765854836,                0],
[                      0,                       0,                       0,        0.98132336139679,                       0,                       0,                       0,      0.0832585170865059,                       0,                       0,      0.0118226371705532,                      0,  1.16082525253296,                 0,                 0,                0],
[                      0,                       0,                       0,                       0,        0.98132336139679,                       0,     -0.0832585170865059,                       0,                       0,     -0.0118226371705532,                       0,                      0,                 0,  1.16082525253296,                 0,                0],
[                      0,                       0,                       0,                       0,                       0,       0.935214638710022,                       0,                       0,                       0,                       0,                       0,                      0,                 0,                 0,  1.34969627857208,                0],
[                      0,                       0,                       0,                       0,    -0.00217520189471543,                       0,        0.70585560798645,                       0,                       0,       0.222776055335999,                       0,                      0,                 0, -1.00350368022919,                 0,                0],
[                      0,                       0,                       0,     0.00217520189471543,                       0,                       0,                       0,        0.70585560798645,                       0,                       0,       0.222776055335999,                      0,  1.00350368022919,                 0,                 0,                0],
[                      0,                       0,                       0,                       0,                       0,                       0,                       0,                       0,                     1.0,                       0,                       0,      0.245607480406761,                 0,                 0,                 0, 5.09571504592896],
[                      0,                       0,                       0,                       0,     -0.0170051399618387,                       0,       -2.30776000022888,                       0,                       0,       0.673351347446442,                       0,                      0,                 0, -7.87802076339722,                 0,                0],
[                      0,                       0,                       0,      0.0170051399618387,                       0,                       0,                       0,       -2.30776000022888,                       0,                       0,       0.673351347446442,                      0,  7.87802076339722,                 0,                 0,                0],
[                      0,                       0,                       0,                       0,                       0,                       0,                       0,                       0,                       0,                       0,                       0,      0.963629364967346,                 0,                 0,                 0, 42.1933479309082],
[ 7.88261662763924*alpha,  6.62846406500116*alpha,  6.62846406500116*alpha,  6.91922911405664*alpha,  6.62846406500116*alpha,  6.62846406500116*alpha,  6.62846406500116*alpha,  6.64070817590793*alpha,  6.62846406500116*alpha,  6.62846406500116*alpha,  6.62959794092468*alpha, 6.62846406500116*alpha,        -1.0*alpha,                 0,                 0,                0],
[ 6.62846406500115*alpha,  7.88261662763923*alpha,  6.62846406500115*alpha,  6.62846406500115*alpha,  6.91922911405664*alpha,  6.62846406500115*alpha,  6.61624252973661*alpha,  6.62846406500115*alpha,  6.62846406500115*alpha,  6.62733038300714*alpha,  6.62846406500115*alpha, 6.62846406500115*alpha,                 0,        -1.0*alpha,                 0,                0],
[ 5.38779349329645*alpha,  5.38779349329645*alpha,  6.40720235639351*alpha,  5.38779349329645*alpha,  5.38779349329645*alpha,  5.61866904984231*alpha,  5.38779349329645*alpha,  5.38779349329645*alpha,  5.38779349329645*alpha,  5.38779349329645*alpha,  5.38779349329645*alpha, 5.38779349329645*alpha,                 0,                 0,        -1.0*alpha,                0],
[0.176727728889814*alpha, 0.176727728889814*alpha, 0.176727728889814*alpha, 0.176727728889814*alpha, 0.176727728889814*alpha, 0.176727728889814*alpha, 0.176727728889814*alpha, 0.176727728889814*alpha, 0.210165872614038*alpha, 0.176727728889814*alpha, 0.176727728889814*alpha, 0.18441171261263*alpha,                 0,                 0,                 0,       -1.0*alpha]])

In [ ]:
# Compute the characteristic polynomial
char_poly = phi_alpha.charpoly().as_expr()

# Expand the characteristic polynomial to separate terms
char_poly_expanded = sp.expand(char_poly)

# Display the results
print("Characteristic Polynomial:")
char_poly_expanded

KeyboardInterrupt: 

In [ ]:
# Factor the polynomial to isolate alpha
factored_poly = sp.collect(char_poly_expanded, alpha)

# Extract the coefficient of alpha and independent terms
P0 = factored_poly.coeff(alpha)
P1 = factored_poly - alpha * P0

# Display the results
print("Factored Polynomial:", factored_poly)
print("Coefficient of alpha:", P0)
print("Independent part:", P1)


# Solve for lambda in both cases
ll = sp.symbols('lambda')  # Unicode lambda
poles_P0 = sp.solve(P0, ll)
print("Poles (terms multiplied by alpha):", poles_P0)


NameError: name 'char_poly_expanded' is not defined

In [ ]:
import sympy as sp

# Define symbolic variables
alpha, s = sp.symbols('alpha s')

# Define the state-space matrices as symbolic
A = sp.Matrix([[2, 1], [-1, -1]])
B = sp.Matrix([[0], [1]])
C = sp.Matrix([[-10, 1]])
D = sp.Matrix([[0]])

# Identity matrix
I = sp.eye(A.shape[0])

# Compute the phi matrix symbolically
exp_A_025 = sp.exp(A * 0.25)
bl = -sp.simplify(sp.inv(C * sp.inv(A) * (exp_A_025 - I) * B) * C * exp_A_025)
br = -sp.eye(1)

# Multiply bl and br by alpha
bl_alpha = alpha * bl
br_alpha = alpha * br

# Construct the augmented phi matrix
phi_alpha = sp.Matrix([[A, B], [bl_alpha, br_alpha]])

# Compute the characteristic polynomial
char_poly = phi_alpha.charpoly(s).as_expr()

# Simplify the characteristic polynomial
char_poly_simplified = sp.simplify(char_poly)

print("Characteristic polynomial of phi_alpha:")
print(char_poly_simplified)


AttributeError: module 'sympy' has no attribute 'inv'